In [ ]:
#!pip install azureml-contrib-fairness
#!pip install fairlearn==0.4.6
#!pip install raiwidgets
#pip install azureml-dataprep[pandas]

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core import *
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace, Dataset, Datastore
import os, shutil
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core import Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace,RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl.utilities import get_primary_metrics
from azureml.data.datapath import DataPath
from azureml.core.datastore import Datastore
from azureml.pipeline.core import InputPortBinding
from azureml.core import Run
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter
from azureml.core import Dataset
from azureml.core import Environment
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.data import OutputFileDatasetConfig
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.core import Run
from azureml.core import get_run
import pandas as pd
import numpy as np
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LGBMExplainableModel
from raiwidgets import FairnessDashboard
from raiwidgets import ExplanationDashboard
from azureml.pipeline.core import Pipeline
from azureml.core import Environment
from azureml.pipeline.core import Pipeline, StepSequence
import datetime

In [ ]:
# Environnement

compute_engine = 'jucalcul1'
cluster_name = 'jucalcul1'
dataset_name = 'trainingdataset'
experiment_name = "25082021"

# if not exist yet
#ws = Workspace.create(name='myworkspace',
#                      subscription_id='<azure-subscription-id>',
#                      resource_group='myresourcegroup',
#                      create_resource_group=True,
#                      location='eastus2')

ws = Workspace.get(subscription_id = subscription_id,
                   resource_group = resource_group,
                   name = workspace_name)

In [ ]:
# Compute 
components_dir = '.'
os.makedirs(components_dir, exist_ok=True)


# Verify that the compute cluster exists
try:
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster : ', cluster_name)
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=3)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex, cluster_name)


In [ ]:
# environment preparation : 
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

aml_run_config = RunConfiguration()
aml_run_config.target = compute_engine

USE_CURATED_ENV = True
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    
    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn'], 
        pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]','pandas','joblib','azureml-core==1.20.0','PyJWT==1.7.1','fairlearn'], 
        pin_sdk_version=False)

In [106]:
# Step 0 environnement
from azureml.data.data_reference import DataReference
data_folder = os.path.join(os.getcwd(), 'data/')
os.makedirs(data_folder, exist_ok=True)
def_blob_store = Datastore.get(ws, 'saving')
ws.set_default_datastore('saving')

#sklearn_env = Environment.from_conda_specification(name = 'sklearn_envv2', file_path = './conda_dependencies.yml')
sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")

# Step 0 data ingestion
raw_ds = Dataset.get_by_name(ws, name=dataset_name, version='latest') 

# specific path for object saving
fileConfig = OutputFileDatasetConfig(name='file_dataset', destination=(def_blob_store, "01-09-2021/last_structured/{run-id}")).as_mount()
dataset_param = PipelineData('dataset')
datadir_param = PipelineData('datadir', is_directory=True)

#eval_datastore_path = DataReference(data_reference_name = "eval_datastore_path", datastore=ws.datastores['saving'], 
#                                                                                        path_on_datastore = "data/eval.csv")
data_ref = def_blob_store.path('3108-v6/{run-id}/model').as_mount()

# Step 1 data preparation
step1 = PythonScriptStep(name = 'prepare data',
                         source_directory = 'scripts',
                         script_name = 'data_prep.py',
                         compute_target = compute_engine,
                         runconfig=aml_run_config,
                         allow_reuse=True,
                         outputs=[dataset_param, datadir_param],
                         arguments = ['--raw-ds', raw_ds.as_named_input('raw_data'), # has_named_input is the method for tabular dataset 
                                      #'--output-dir', fileConfig
                                      '--dataset', dataset_param, 
                                      '--datadir', datadir_param
                                      ])

# register clean data preprocess
step1_output_ds = fileConfig.register_on_complete(name='processed_data', 
                                                        description = 'files from step1')
model_file = PipelineData("model_file", datastore=def_blob_store,
                                        pipeline_output_name= 'model_output',
                                        training_output=TrainingOutput(type='Model'))

# Step 2 : training 
train_src = ScriptRunConfig(source_directory = 'scripts',
                            script = 'model_script2.py',
                            compute_target = compute_engine,
                            arguments = ["--input-dir", fileConfig,
                                         #'--eval_file', str(data_ref),
                                         '--dataset', dataset_param, 
                                         '--datadir', datadir_param
                                         ],
                            environment = sklearn_env)


# Step 2 : hyperparameter 
param_sampling = RandomParameterSampling( {
    "--n_estimators": choice(15, 50, 100, 200, 300),
    "--criterion": choice("gini", "entropy"),
    "--max_depth": choice(2, 8, 16)
    }
)

hd_config = HyperDriveConfig(run_config=train_src, 
                             hyperparameter_sampling=param_sampling,
                             primary_metric_name='precision', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=2, # small nb for testing 
                             max_concurrent_runs=2)


    
hd_step = HyperDriveStep(
    allow_reuse=True,
    inputs=[dataset_param, datadir_param],
    name='hyperparameters',
    outputs = [model_file],
    hyperdrive_config=hd_config)



# step 3 : register best model
register_model = PythonScriptStep(name = 'Model Registration',
                                  script_name = 'scripts/register_model2.py',
                                  compute_target = compute_engine,
                                  runconfig=aml_run_config,
                                  allow_reuse=True
                                  )


# step 4 : publish pipeline
pipeline_training = [step1, hd_step, register_model]
pipeline = Pipeline(workspace=ws, steps =StepSequence(steps=pipeline_training))
pipeline_run = pipeline.submit(experiment_name, regenerate_outputs=True)

published_pipeline1 = pipeline.publish(
                        name="Template_Pipeline_Notebook",
                        description="Published Pipeline Description",
                        version="1.0")


# step 5 : schedule pipeline run every day
#recurrence = ScheduleRecurrence(frequency='Day', interval=1)
#recurring_schedule = Schedule.create(ws, name='DailySchedule', 
#                            description='Once a day',
#                            pipeline_id=published_pipeline1.id, 
#                            experiment_name= experiment_name, 
#                            recurrence=recurrence)

Created step prepare data [b811b71d][be3db780-0009-4ece-acae-c13c34ac9351], (This step will run and generate new outputs)
Created step hyperparameters [e1207a59][c6017cea-5081-4d07-9a67-58fbf960ed4c], (This step will run and generate new outputs)
Created step Model Registration [d5b62edb][b0558bc1-ed14-41bd-b87b-acf83767b543], (This step will run and generate new outputs)
Submitted PipelineRun bec4b943-7574-4081-8f44-6faf41634129
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/bec4b943-7574-4081-8f44-6faf41634129?wsid=/subscriptions/a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b/resourcegroups/ressourcesecu/workspaces/deuxiemeespace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


In [102]:
pipeline_run.get_details()['properties']
summaryid = [0]
summarymetric = [0]

for child_run in pipeline_run.get_children():
    listid = child_run.id
    print('id parent', listid)
    for  child_run2 in child_run.get_children():
        print('id pipeline step',child_run2)
        for  child_run3 in child_run2.get_children():
            listid = child_run3.id
            summaryid.append(listid)
            list_metric = child_run3.get_metrics()['Test Precison']
            summarymetric.append(list_metric)

print(summarymetric)
print(summaryid)
maxi = summarymetric.index(max(summarymetric))
maxi_id = summaryid[maxi]
print('best model id : ', maxi_id)


print('start register model')
for child_run3 in child_run2.get_children():
    if child_run3.id == maxi_id:
        #child_run3.register_model(model_name='rf_tuning', model_path='outputs_here/model.pkl')
        print("ok it's saved")
        metrics = child_run3.get_metrics()
        df = pd.DataFrame(list(metrics.items()),columns = ['metrics','value']) 
        print(df)

id parent c8f97c3b-e01d-4b15-b92c-c808f865cc87
[0]
[0]
best model id :  0
start register model


In [39]:
for child_run3 in child_run2.get_children():
    if child_run3.id == maxi_id:
        #child_run3.register_model(model_name='rf_tuning', model_path='outputs_here/model.pkl')
        #print("ok it's saved")
        print(child_run3.get_metrics())
        print(child_run3.get_details())

{'Split criterion': 'entropy', 'Test Accuracy2': 0.9033333333333333, 'Test Accuracy': 0.9033333333333333, 'Test Recall': 0.9161248374512354, 'Test Precison': 0.9501011463250169, 'Test F1 Score': 0.9328037073816617}
{'runId': 'HD_eaf28131-56cd-403a-ad2f-162e00e99978_1', 'target': 'jucalcul1', 'status': 'Completed', 'startTimeUtc': '2021-08-31T17:55:56.30267Z', 'endTimeUtc': '2021-08-31T17:57:33.524442Z', 'properties': {'azureml.moduleid': '070a421c-0944-43e0-beb3-f323e70699f9', '_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': 'af186d4d-7df8-4b2e-a448-0b0804124c3d', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [{'dataset': {'id': '50bcac87-f5e5-4298-9ede-2938383a87d8'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'input_fd1590ff', 'mechanism': 'Mount'}}], 'outputDatasets': [], 'runDefinition': {'script': 'model_script2.py', 'command': '', 'useAbsolutePath': False, 'arguments': [